# Comprehensive Data Exploration

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
os.listdir("../input/house-prices-advanced-regression-techniques")

## Import library

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

## Bring the Data

In [ ]:
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
df_train.columns

In [ ]:
#descriptive statistics summary
df_train['SalePrice'].describe()

In [ ]:
#histogram
sns.displot(df_train['SalePrice'])

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % df_train['SalePrice'].skew())
print("Kurtosis: %f" % df_train['SalePrice'].kurt())

# Analysis of the numerical variables

In [ ]:
#scatter plot grlivarea(지상에 있는 면적 크기)/saleprice
var = "GrLivArea"
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0, 800000))


#### SalePrice와 GrLivArea => 선형 관계

In [ ]:
#scatter plot totalbsmtsf/saleprice
var = "TotalBsmtSF"
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0, 800000))

#### SalePrice와 TotalBsmtSF => 지수 관계, 몇몇 TotalBsmtSF는 0에 가까운 값을 가지고 있다

# Analysis of the categorical features

In [ ]:
#box plot overallqual(전반적인 퀄리티)/saleprice
var = "OverallQual"
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
fig, ax = plt.subplots(figsize=(8, 6))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000)

#### OverallQual이 SalePrice에 많은 영향을 미침

In [ ]:
var = "YearBuilt"
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
fig, ax = plt.subplots(figsize=(16, 8))
fig = sns.boxplot(x=var, y="SalePrice", data=data)
fig.axis(ymin=0, ymax=800000)
plt.xticks(rotation=90)

#### YearBuilt는 SalePrice와 강력한 관계는 없지만 새로운 연식이 가격이 높은 편이다.

# Correlation matrix(heatmap style)

In [ ]:
#correlation matrix
corrmat = df_train.corr()
fig, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True)

#### SalePrice의 상관관계 파악

# 'SalePrice' correlation matrix(zoomed heatmap style)

In [ ]:
#saleprice correlation matrix
k = 10 #number of variables for heatmap
cols = corrmat.nlargest(k, "SalePrice")['SalePrice'].index
cm = np.corrcoef(df_train[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar = True, annot=True, square=True, fmt='.2f',
                  annot_kws={'size':10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

#### GarageCars, GarageArea => 거의 똑같은 의미 => SalePrice와 상관 관계가 높은 GarageCars 사용
#### TotalBsmtSF, 1stFloor => TotalBsmtSF만 사용

# 관련 높은 변수들 사이의 scatter plots

In [ ]:
#scatterplot
sns.set()
cols = ['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']
sns.pairplot(df_train[cols], size=2.5)
plt.show()

# Missing data

In [ ]:
#missing data
total = df_train.isnull().sum().sort_values(ascending=False) #missing data 개수
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
#isnull().count() null값을 포함해서 카운트
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

#### missing data 처리
* 데이터의 15% 이상이 누락되었을 때에는 해당 변수 삭제 (PoolQC, MiscFeature, Alley 등)
* GarageX 변수들은 GarageCars를 제외하고 전부 삭제
* BsmtX도 마찬가지
* MasVnrArea, MasVnrType(벽돌 면적, 유형)은 YearBuilt, OverallQual과 관계 多 -> 삭제
* Electrical은 하나의 missing data => 해당 데이터 삭제

In [ ]:
#dealing with missing data
df_train = df_train.drop((missing_data[missing_data['Total'] > 1]).index, 1) #electrical 변수를 제외하고 missing data 있는 변수 삭제
df_train = df_train.drop(df_train.loc[df_train['Electrical'].isnull()].index)
df_train.isnull().sum().max()

# Outliers

### Univariate analysis

In [ ]:
#standardizing data
saleprice_scaled = StandardScaler().fit_transform(df_train['SalePrice'][:, np.newaxis])
low_range = saleprice_scaled[saleprice_scaled[:, 0].argsort()][:10]
high_range = saleprice_scaled[saleprice_scaled[:, 0].argsort()][-10:]
print('outer range (low) of the distribution: ')
print(low_range)
print('\nouter range (high) of the distribution: ')
print(high_range)

- Low range: low range 값들은 유사하며 0과 가깝다.
- High range: 0에서 멀리 떨어져 있고, 7.xx 값들은 범위가 많이 벗어나 있다.

### Bivariate analysis

In [ ]:
#bivariate analysis saleprice/grlivarea
var = 'GrLivArea'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x=var, y='SalePrice', ylim=(0, 800000))

- 7.xx는 추세를 따르고 있으므로 유지
- 오른쪽 두 값은 클러스터에서 벗어나있음. 규칙을 벗어나므로 outlier로 정의하고 삭제 -> 아마 농경지

In [ ]:
df_train.sort_values(by = 'GrLivArea', ascending = False)[:2]

In [ ]:
#deleting data
df_train = df_train.drop(df_train[df_train['Id'] == 1299].index)
df_train = df_train.drop(df_train[df_train['Id'] == 524].index)

In [ ]:
#bivariate analysis saleprice/grlivarea
var = 'TotalBsmtSF'
data = pd.concat([df_train['SalePrice'], df_train[var]], axis=1)
data.plot.scatter(x = var, y = 'SalePrice', ylim=(0, 800000))

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train['SalePrice'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

In [ ]:
#applying log transformation
df_train['SalePrice'] = np.log(df_train['SalePrice'])

In [ ]:
#transformed histogram and normal probability plot
sns.distplot(df_train['SalePrice'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['SalePrice'], plot=plt)

In [ ]:
#histogram and normal probability plot
sns.distplot(df_train['GrLivArea'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

In [ ]:
#data transform
df_train['GrLivArea'] = np.log(df_train['GrLivArea'])

In [ ]:
#transformed
sns.distplot(df_train['GrLivArea'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['GrLivArea'], plot=plt)

In [ ]:
sns.distplot(df_train['TotalBsmtSF'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train['TotalBsmtSF'], plot=plt)

- 0값이 많다. 그러나 0은 로그 변환을 허용하지 않는다. => 로그 변환을 위해 지하실 여부로 binary variable 생성

In [ ]:
df_train['HasBsmt'] = pd.Series(len(df_train['TotalBsmtSF']), index = df_train.index)
df_train['HasBsmt'] = 0
df_train.loc[df_train['TotalBsmtSF']>0, 'HasBsmt'] = 1

In [ ]:
df_train.loc[df_train['HasBsmt']>0, 'TotalBsmtSF'] = np.log(df_train['TotalBsmtSF'])

In [ ]:
sns.distplot(df_train.loc[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm)
fig = plt.figure()
res = stats.probplot(df_train.loc[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

In [ ]:
#scatter plot
plt.scatter(df_train['GrLivArea'], df_train['SalePrice'])

In [ ]:
plt.scatter(df_train[df_train['TotalBsmtSF']>0]['TotalBsmtSF'], df_train[df_train['TotalBsmtSF']>0]['SalePrice'])

In [ ]:
#categorical 변수를 더미로 변경
df_train = pd.get_dummies(df_train)

# Data preprocessing